In [1]:
import sys
print(sys.version)

3.8.2 | packaged by conda-forge | (default, Apr 24 2020, 08:20:52) 
[GCC 7.3.0]


In [2]:
import pytorch_lightning
pytorch_lightning.__version__

'0.7.6'

In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
sys.path.append("/nfs/homedirs/herrmanp/clevertest/adversarial-robustness-toolbox")
import art

In [5]:
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_lightning import loggers
import config


from classification.models.M5 import M5, M5PLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule

from utils.RobustnessExperiment import load_module
from utils.Visual import show_audio, show_spectogram, show_waveplot

/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [6]:
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

# Train Model

In [7]:
from pytorch_lightning.callbacks import Callback

class SaveCallback(Callback): 
    def __init__(self, model_name = "newest_model"):
        super().__init__()
        self.model_name = model_name
        self.best_val_acc = None

    def on_epoch_end(self, trainer, pl_module):
        if not self.best_val_acc or pl_module.val_results_history[-1]["val_acc"] > self.best_val_acc:
            print("new best val acc", pl_module.val_results_history[-1]["val_acc"])
            self.best_val_acc = pl_module.val_results_history[-1]["val_acc"]
            save_path = self.model_name + "_v{}".format(trainer.logger.version) +  "_best.p"
            pl_module.save(save_path, overwrite_if_exists=True)
            print("Saved checkpoint at epoch {} at \"{}\"".format((trainer.current_epoch + 1), save_path))

cb = SaveCallback("iiii")

In [8]:
# best M5: batchsize 32, lr 9e-4, weithgdecay 0.001, lr decay 0.85
import config
import pytorch_lightning as pl
from classification.models.M5 import M5, M5PLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule
from attacks.pgd import ProjectedGradientDescent

print("Currently Best:", cb.best_val_acc)

hparams = {
    "batch_size": 64,
    "learning_rate": 9e-4,
    "weight_decay": 1e-3,
    "lr_decay": 0.85,
    "epochs": 100
}

model = M5PLModule(hparams)
model.prepare_data()
datasetHandler.load(model, 'training')
datasetHandler.load(model, 'validation')

# to train with gaussian noise:
model.set_smooth(50)

trainer = pl.Trainer(
    max_epochs=hparams["epochs"],
    logger= loggers.TensorBoardLogger(config.LOG_DIR, name=type(model).__name__),
    gpus=1 if torch.cuda.is_available() else None,
    callbacks=[cb]
)

trainer.fit(model)

Currently Best: None
Loading cached train data from /nfs/students/summer-term-2020/project-4/data/data_8k
Loading cached val data from /nfs/students/summer-term-2020/project-4/data/data_8k


GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
Set SLURM handle signals.

   | Name           | Type         | Params
--------------------------------------------
0  | model          | M5           | 555 K 
1  | model.model    | Sequential   | 555 K 
2  | model.model.0  | Conv1d       | 10 K  
3  | model.model.1  | BatchNorm1d  | 256   
4  | model.model.2  | MaxPool1d    | 0     
5  | model.model.3  | Dropout      | 0     
6  | model.model.4  | Conv1d       | 49 K  
7  | model.model.5  | BatchNorm1d  | 256   
8  | model.model.6  | MaxPool1d    | 0     
9  | model.model.7  | Dropout      | 0     
10 | mod

<class 'dict'>


AttributeError: 'dict' object has no attribute 'float'

In [ ]:
!ls

In [9]:
#model = load_module("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/SpectrogramCNN/optuna_90.5.p", SpectrogramCNNPLModule)
#model = load_module("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_vanilla.p", M5PLModule)
model = load_module("/nfs/homedirs/herrmanp/shortcut/SAVED_MODELS/M5/M5_attack_num_iters_v99.p", M5PLModule)
model.prepare_data()
datasetHandler.load(model, 'training')
datasetHandler.load(model, 'validation')
model.cuda()
model.report()

AttributeError: 'dict' object has no attribute 'float'

In [10]:
x_test = model.val_dataloader().dataset[0:100]
x_test = []
y_test = []
for i in range(100):
    x_test.append(model.val_dataloader().dataset[i]["audio"])
    y_test.append(model.val_dataloader().dataset[i]["label"])


In [11]:
import torch.nn.functional as F

from art.estimators.classification.pytorch import PyTorchClassifier
modelX = PyTorchClassifier(model =  model.model,
        loss = F.cross_entropy,
        input_shape = (1, 8000),
        nb_classes = 2,
        channels_first = True)

In [12]:
scores = []
for i in range(10):
    x = model.val_dataloader().dataset[i]["audio"].numpy()
    out = art.metrics.clever_u(classifier = modelX, x = x, nb_batches = 50, batch_size = 5, radius = 1, norm = np.inf, c_init = 1.0, pool_factor = 100)
    print(out)
    scores.append(out)
print("AVG OVER 10 SAMPLES: {}".format(sum(scores)/10))

CLEVER untargeted:   0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [13]:
art.metrics.clever_t(classifier = modelX, target_class = 0, x = x, nb_batches = 50, batch_size = 5, radius = 1, norm = np.inf, c_init = 1.0, pool_factor = 100)

ValueError: The targeted class is the predicted class.

In [19]:
x_test_arr = [t.numpy() for t in x_test]
y_test_arr = [t for t in y_test]


total = np.vstack(x_test_arr)
totaly = np.array(y_test_arr)
print(total.shape)

b = np.zeros((totaly.size, totaly.max()+1))
b[np.arange(totaly.size),totaly] = 1


(100, 80000)


In [24]:
b.shape

(100, 2)

In [15]:
from art.attacks.evasion import ProjectedGradientDescentPyTorch

In [16]:
from art.defences import AdversarialTrainer


ImportError: cannot import name 'AdversarialTrainer' from 'art.defences' (/nfs/homedirs/herrmanp/clevertest/adversarial-robustness-toolbox/art/defences/__init__.py)

In [17]:
x = total.astype(np.float)
print(x)
print(totaly.shape)

[[-0.2246148  -0.12731266 -0.11558467 ... -0.02509868  0.02699089
  -0.00840473]
 [-0.006051    0.04431272  0.02741277 ...  0.          0.
   0.        ]
 [-0.00947881 -0.00280213 -0.02831542 ...  0.14356637  0.02705276
  -0.25976855]
 ...
 [-0.20915544  0.77309835  0.92083621 ... -0.30089265  0.17747772
   0.16884112]
 [-0.00476503 -0.22795105 -0.13065314 ... -0.03364056 -0.03021657
  -0.02279794]
 [-0.02124894 -0.01703423 -0.01570475 ...  0.54430318  0.42404008
   0.37150085]]
(100,)


In [26]:
pgd = ProjectedGradientDescentPyTorch(modelX, eps=8, eps_step=2, max_iter=10, num_random_init=20)
x_test_pgd = pgd.generate(total.astype(np.double))



TypeError: super(type, obj): obj must be an instance or subtype of type

### Run a single attack

In [ ]:
from attacks.volume import VolumeAttack
from attacks.pitch import PitchAttack
from attacks.speed import TimeStretchAttack
from attacks.pgd import ProjectedGradientDescent

attack = VolumeAttack(model.model, model.val_dataloader(), {"epsilon":0.9}, early_stopping=-1, device='cuda', save_samples=True)

attack.attack()
attack.report( )

In [ ]:
attack.showAdversarialExample(target_class=1)

* change:
    * return accuracy
    * make easy to compare via spectogram & waveform
